In [8]:
import yfinance as yf
import pandas as pd

# yfinance를 사용하여 2010-01-01부터 2023-08-31까지의 KOSPI200 데이터를 가져옵니다.
data = yf.download('^KS11', start='2010-01-01', end='2023-08-31')

# 'Adj Close' 컬럼만 사용합니다.
kospi = data[['Adj Close']].reset_index()

# kospi의 Date를 인덱스로 설정합니다.
kospi.set_index('Date', inplace=True)

# 새로운 DataFrame을 생성합니다.
result = pd.DataFrame(columns=['base_date', 'm_return', 'kospi', 'before_last'])

# 각 월의 마지막 영업일만 선택합니다.
kospi_month_end = kospi.resample('M').last()

# 각 월의 두 번째로 마지막 영업일을 선택합니다.
kospi_before_last = kospi.resample('M').apply(lambda x: x.index[-2] if len(x) > 1 else None).iloc[:, 0]

for i in range(len(kospi_month_end)):
    base_date = kospi_month_end.index[i]
    kospi_val = round(kospi_month_end.iloc[i]['Adj Close'], 3)
    
    # before_last 값을 찾습니다.
    if i == 0:
        before_last = None  
        m_return = None  
    else:
        # 이전 행의 데이터를 참조합니다.
        before_last = kospi_before_last[i-1].strftime('%Y-%m-%d')
        
        # 이전 before_last 값을 참조하여 수익률을 계산합니다: (이번 달 kospi - 지난 달 kospi) / 지난 달 kospi
        before_last_val = kospi.loc[kospi_before_last[i-1]]['Adj Close']
        m_return = round((kospi_val - before_last_val) / before_last_val, 3)
    
    # 결과를 저장합니다.
    new_row = {
        'base_date': base_date.strftime('%Y-%m-%d'),
        'm_return': m_return,
        'kospi': kospi_val,
        'before_last': before_last
    }
    result = pd.concat([result, pd.DataFrame([new_row])], ignore_index=True)

# DataFrame을 확인하거나 CSV로 저장할 수 있습니다.
result.to_csv('kospi_month_end.csv', index=False)


[*********************100%***********************]  1 of 1 completed


C:\Users\성현태\AppData\Local\Temp\ipykernel_2796\2786280689.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, pd.DataFrame([new_row])], ignore_index=True)
C:\Users\성현태\AppData\Local\Temp\ipykernel_2796\2786280689.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  before_last = kospi_before_last[i-1].strftime('%Y-%m-%d')
C:\Users\성현태\AppData\Local\Temp\ipykernel_2796\2786280689.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (

In [9]:
import pandas as pd

# 파일 읽기
kospi_month_end = pd.read_csv('kospi_month_end.csv')
kospi_month_end.head()


,base_date,m_return,kospi,before_last
0,2010-01-31,NaN,1602.43,NaN
1,2010-02-28,-0.029,1594.58,2010-01-28
2,2010-03-31,0.066,1692.85,2010-02-25
3,2010-04-30,0.024,1741.56,2010-03-30
4,2010-05-31,-0.050,1641.25,2010-04-29


In [10]:
# 'before_last' 컬럼의 값을 한 행 위로 옮깁니다.
kospi_month_end['before_last'] = kospi_month_end['before_last'].shift(-1)

# 결과 확인
kospi_month_end.head()
kospi_month_end.to_csv('real.csv', index=False)
